In [1]:
# Default env type is test
# Change this when running on colab or kaggle
ENV_TYPE = "TEST"

In [ ]:
# Important deps
!apt update && apt install -y libgl1-mesa-glx imagemagick

if ENV_TYPE != "TEST":
    !apt install -y git aria2
    # Clone the repo
    !git clone https://github.com/kk-digital/kcg-ml-sd1p4.git --recurse-submodules -b lora
    %cd kcg-ml-sd1p4
    # Download model weights
    !aria2c https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -o ./input/model/sd-v1-4.ckpt -j 10

In [ ]:
# Install reqs
!pip3 install -r lora-tuning/requirements.txt

In [ ]:
# Let's now run the 'test suite'
!pip3 install accelerate==0.19.0
!git pull
if ENV_TYPE != "TEST":
    !python3 lora-tuning/lora_train.py --num_repeats 1 --lowram True
else:
    !python3 lora-tuning/lora_train.py --num_repeats 1

In [ ]:
# Shuffle up dependencies to fix inference
!pip3 install -r lora-tuning/inference/requirements.txt

In [ ]:
# Generate images for each epoch:
import os
import subprocess

# Set the directory containing the .safetensors files
lora_path = "./output/LoRa/Test/output/"
checkpoint_path = "input/model/sd-v1-4.ckpt"
scale = 512
prompt = "a sketch of a pixwaifu, cute, small, chibi, DeviantArt trending"

# Get a list of all .safetensors files in the directory
file_list = [filename for filename in os.listdir(lora_path) if filename.endswith(".safetensors")]

# Set up epoch to begin counting
epoch = 1

for filename in file_list:
    # Generate the image using txt2img.py for each .safetensors file
    output_filename = os.path.splitext(filename)[0] + ".png"
    command = f"python3 lora-tuning/inference/txt2img.py --checkpoint_path {checkpoint_path} --lora {os.path.join(lora_path, filename)} --output {output_filename} --scale {scale} --prompt \"{prompt}\""
    !{command}

    epoch += 1

# Generate a tiled image with all of the epochs
!montage -label '%t' *.png -thumbnail 512x512! -pointsize 22 -gravity center \
    -background Transparent -geometry +0+0 -tile 3x results_lora_epochs.png